# Linear Regression Analysis of Cigarette Data

This notebook performs a linear regression analysis on cigarette data. The analysis includes:

1. Loading and preparing the data.
2. Building a regression model with multiple predictors.
3. Computing and interpreting the model coefficients.
4. Performing an ANOVA to evaluate the significance of the regression model.
5. Evaluating the model fit with the coefficient of determination (R²).
6. Analyzing the correlation between predictor variables.
7. Building a reduced regression model with fewer predictors and comparing it with the original model.

In [73]:
# Import necessary libraries
import numpy as np
import pandas as pd
from scipy.stats import f

## 1. Loading and Preparing the Data

We start by loading the data from an Excel file and selecting the relevant columns. The columns are renamed for simplicity.

In [74]:
# Load the data
file_path = 'TutorialCigarette.xlsx'
df = pd.read_excel(file_path, header=1, sheet_name="Feuil1", usecols=['Tar (mg)', 'Nicotine (mg)', 'Weight (g)', 'Carbon Monoxide(mg)'])

# Renaming the columns to simplify
df.rename(columns={'Tar (mg)': 'tar', 'Nicotine (mg)': 'nicotine', 'Weight (g)': 'weight', 'Carbon Monoxide(mg)': 'carbon_monoxide'}, inplace=True)

# Display the first few rows of the data
df.head()

,tar,nicotine,weight,carbon_monoxide
0,14.1,0.86,0.9853,13.6
1,16.0,1.06,1.0938,16.6
2,29.8,2.03,1.1650,23.5
3,8.0,0.67,0.9280,10.2
4,4.1,0.40,0.9462,5.4


## 2. Building the Regression Model

We'll select the predictor variables and add an intercept. Then we'll fit the regression model using linear algebra.

In [75]:
# Selecting the predictor variables
X = df[['weight', 'tar', 'nicotine']]

# Adding a column of ones to account for the intercept (β4)
X['intercept'] = 1
X = X[['intercept', 'weight', 'tar', 'nicotine']]

# Converting the DataFrame to a NumPy array for later matrix operations
X_matrix = X.values
y = df['carbon_monoxide'].values

# Calculating the regression coefficients using the normal equation
X_transpose = X.T
beta = np.linalg.inv(X_transpose.dot(X)).dot(X_transpose).dot(y)
beta

array([ 3.20219002, -0.13048185,  0.96257386, -2.63166111])

## 3. Making Predictions

We'll use the model coefficients to predict the values of the response variable.

In [76]:
# Predicting values of y based on the regression model
y_pred = X.dot(beta)
y_pred

0     14.382689
1     15.671090
2     26.392608
3      9.018481
4      5.972616
5     14.787937
6      9.538812
7     12.517658
8     16.111168
9     14.744665
10    13.605650
11    15.247003
12     9.083587
13    11.976175
14     9.806794
15     3.720207
16    16.130192
17    12.545305
18    15.759552
19     6.309658
20    14.370138
21     8.495715
22     9.538003
23    15.025113
24    12.449183
dtype: float64

## 4. Performing ANOVA

We compute the Sum of Squares for Total (SST), Regression (SSR), and Error (SSE). Then we calculate the F-statistic and the corresponding p-value. Finally, we construct the ANOVA table.


In [77]:
# Given values
n = len(y)  # Number of observations
p = X.shape[1] - 1  # Number of predictors, excluding the intercept

# Computing SST, SSR, and SSE
SST = np.sum((y - np.mean(y)) ** 2)
SSR = np.sum((y_pred - np.mean(y)) ** 2)
SSE = np.sum((y - y_pred) ** 2)

# Calculating the degrees of freedom
dfR = p
dfE = n - p - 1
dfT = n - 1

# Mean Squares
MSR = SSR / dfR
MSE = SSE / dfE

# F-statistic
F = MSR / MSE
P_value = f.sf(F, dfR, dfE)  # This computes the p-value for the F-statistic

# Constructing the ANOVA table
anova_table = pd.DataFrame({
    "Source": ["Regression", "Residual", "Total"],
    "Sum of Squares": [SSR, SSE, SST],
    "df": [dfR, dfE, dfT],
    "Mean Square": [MSR, MSE, np.nan],
    "F": [F, np.nan, np.nan],
    "P-value": [P_value, np.nan, np.nan]
})

anova_table

,Source,Sum of Squares,df,Mean Square,F,P-value
0,Regression,495.257814,3,165.085938,78.983834,1.328810e-11
1,Residual,43.892586,21,2.090123,NaN,NaN
2,Total,539.150400,24,NaN,NaN,NaN


## 5. Model Evaluation

We'll compute the coefficient of determination (R²) to evaluate the goodness of fit of the model.


In [78]:
# Coefficient of determination (R²)
R_squared = 1 - (SSE / SST)
R_squared

np.float64(0.9185893479475058)

## 6. Correlation Analysis

We'll compute and display the correlation matrix for the predictor variables.


In [79]:
# Compute the correlation matrix
predictors = df[['weight', 'tar', 'nicotine']]
correlation_matrix = predictors.corr()

correlation_matrix

,weight,tar,nicotine
weight,1.000000,0.490765,0.500183
tar,0.490765,1.000000,0.976608
nicotine,0.500183,0.976608,1.000000


## 7. Building a Reduced Model

We'll create a reduced model using only `weight` and `tar` as predictors, and perform similar analysis as with the full model.


In [80]:
# Selecting the predictors for the reduced model and the response variable
X_reduced = df[['weight', 'tar']].copy()
X_reduced.loc[:, 'intercept'] = 1  # Adding the intercept term

# Converting to a NumPy array for matrix operations
X_reduced = X_reduced[['intercept', 'weight', 'tar']].values

# Reusing the dependent variable
y = df['carbon_monoxide'].values

# Calculating the regression coefficients for the reduced model
beta_reduced = np.linalg.inv(X_reduced.T.dot(X_reduced)).dot(X_reduced.T).dot(y)

# Predicting values for the reduced model
y_pred_reduced = X_reduced.dot(beta_reduced)

# Calculating sums of squares for the reduced model
SST_reduced = np.sum((y - np.mean(y)) ** 2)
SSR_reduced = np.sum((y_pred_reduced - np.mean(y)) ** 2)
SSE_reduced = np.sum((y - y_pred_reduced) ** 2)

# Degrees of freedom for the reduced model
n = len(y)
p_reduced = 2  # Number of predictors in the reduced model
dfR_reduced = p_reduced
dfE_reduced = n - p_reduced - 1
dfT_reduced = n - 1

# Mean Squares for the reduced model
MSR_reduced = SSR_reduced / dfR_reduced
MSE_reduced = SSE_reduced / dfE_reduced

# F-statistic for the reduced model
F_reduced = MSR_reduced / MSE_reduced

# P-value for the F-statistic of the reduced model
P_value_reduced = f.sf(F_reduced, dfR_reduced, dfE_reduced)

# ANOVA Table for the reduced model
anova_table_reduced = pd.DataFrame({
    "Source": ["Regression", "Residual", "Total"],
    "Sum of Squares": [SSR_reduced, SSE_reduced, SST_reduced],
    "df": [dfR_reduced, dfE_reduced, dfT_reduced],
    "Mean Square": [MSR_reduced, MSE_reduced, np.nan],
    "F": [F_reduced, np.nan, np.nan],
    "P-value": [P_value_reduced, np.nan, np.nan]
})

anova_table_reduced

,Source,Sum of Squares,df,Mean Square,F,P-value
0,Regression,494.306381,2,247.153190,121.250733,1.318076e-12
1,Residual,44.844019,22,2.038365,NaN,NaN
2,Total,539.150400,24,NaN,NaN,NaN
